# Bug: run thành công ở 
```
1. Vào airflow web server: localhost:8080
- Đăng nhập với tài khoản admin và mật khẩu admin
- Run workflow, nó sẽ tự động click vào DAG `dags/kafka-stream` và chạy luồng để kéo dữ liệu từ API vào Kafka

2. vào Control Center: localhost:9021
- Check topic `users_created`
- Check data đã được đẩy lên Kafka
```bash
docker exec -it broker kafka-console-consumer
 --bootstrap-server broker:29092 --topic users_created --from-beginning
``` 

```
tại sao check bằng
bash
docker exec -it broker kafka-console-consumer
 --bootstrap-server broker:29092 --topic users_created --from-beginning


thấy dataset rồi 
mà sao trên http://localhost:9021/clusters/_B-EUdw9RSKFvCU2fGgr_Q/management/topics/users_created/message-viewer

lại báo: no message
```

Nếu bạn thấy dữ liệu hiển thị khi kiểm tra bằng lệnh CLI nhưng không thấy dữ liệu trên giao diện **Control Center** tại `http://localhost:9021`, điều này có thể do một số nguyên nhân sau:

---

### **Nguyên nhân tiềm năng**

#### 1. **Control Center không kết nối đúng với Kafka Broker**
   - **Control Center** có thể không được cấu hình đúng để kết nối với Kafka Broker tại `broker:29092`.
   - Điều này dẫn đến việc **Control Center** không thể lấy được dữ liệu từ Kafka, mặc dù dữ liệu thực sự tồn tại trên Kafka Broker.

   **Cách kiểm tra:**
   - Kiểm tra log của Control Center:
     ```bash
     docker logs control-center
     ```
   - Xem log có báo lỗi kết nối tới Kafka hoặc `bootstrap.servers` không chính xác không.

---

#### 2. **Không có quyền truy cập vào topic**
   - **Control Center** có thể không được cấu hình để xem dữ liệu từ topic `users_created`.
   - Topic có thể không được gắn với một ACL (Access Control List) mà Control Center có thể đọc được.

   **Cách kiểm tra:**
   - Xác minh ACL trên topic bằng cách chạy lệnh:
     ```bash
     docker exec -it broker kafka-acls --list --bootstrap-server broker:29092
     ```
   - Đảm bảo rằng Control Center có quyền đọc topic `users_created`.

---

#### 3. **Dữ liệu trong Kafka đã bị phân vùng lại**
   - Nếu dữ liệu trong topic `users_created` bị phân phối không đồng đều giữa các partition, Control Center có thể không đọc được một số partition.

   **Cách kiểm tra:**
   - Kiểm tra số lượng partition của topic:
     ```bash
     docker exec -it broker kafka-topics --describe --topic users_created --bootstrap-server broker:29092
     ```
   - Đảm bảo rằng tất cả partition được phân phối hợp lý và có leader hoạt động.

---

#### 4. **Control Center không hỗ trợ dữ liệu kiểu `Message Viewer` trong giao diện**
   - Có khả năng **Control Center** không hiển thị đúng dữ liệu từ topic nếu dữ liệu không được lưu trữ theo format mà giao diện `Message Viewer` hỗ trợ (JSON, Avro, etc.).
   - Dữ liệu có thể vẫn tồn tại nhưng không hiển thị được do format không đúng.

   **Cách kiểm tra:**
   - Kiểm tra format dữ liệu bằng lệnh CLI:
     ```bash
     docker exec -it broker kafka-console-consumer \
       --bootstrap-server broker:29092 --topic users_created --from-beginning
     ```
   - Nếu dữ liệu không phải JSON hoặc không được định dạng rõ ràng, bạn có thể gặp vấn đề hiển thị trên giao diện.

---

#### 5. **Lỗi caching hoặc refresh trong Control Center**
   - **Control Center** đôi khi không tự động làm mới dữ liệu trên giao diện web. Điều này có thể khiến bạn không thấy dữ liệu mới.

   **Cách xử lý:**
   - Thử **refresh** giao diện bằng cách:
     - Click vào nút `Refresh` trong giao diện `Message Viewer`.
     - Hoặc thử load lại trang.

---

#### 6. **Offset của Consumer (Control Center) không đúng**
   - Control Center có thể không tiêu thụ dữ liệu từ đầu (beginning) của topic.

   **Cách kiểm tra:**
   - Đảm bảo rằng consumer của Control Center đang đọc từ offset `earliest`:
     - Kiểm tra offset hiện tại của topic:
       ```bash
       docker exec -it broker kafka-consumer-groups --bootstrap-server broker:29092 --describe --group <consumer-group-id>
       ```

---

### **Cách xử lý tổng hợp**

1. **Kiểm tra kết nối Control Center với Kafka Broker:**
   - Đảm bảo `CONTROL_CENTER_BOOTSTRAP_SERVERS` được cấu hình chính xác trong `docker-compose.yml`:
     ```yaml
     CONTROL_CENTER_BOOTSTRAP_SERVERS: 'broker:29092'
     ```
   - Nếu có sửa đổi, khởi động lại Control Center:
     ```bash
     docker-compose restart control-center
     ```

2. **Kiểm tra topic trong Control Center:**
   - Vào mục `Topics` trong Control Center, kiểm tra xem topic `users_created` có hiển thị hay không.
   - Nếu không thấy, thử tạo lại topic với quyền phù hợp:
     ```bash
     docker exec -it broker kafka-topics --create --topic users_created --bootstrap-server broker:29092 --partitions 1 --replication-factor 1
     ```

3. **Kiểm tra log Kafka và Control Center:**
   - Xem log của Kafka Broker và Control Center để phát hiện lỗi:
     ```bash
     docker logs broker
     docker logs control-center
     ```

4. **Kiểm tra dữ liệu trực tiếp từ Kafka:**
   - Dùng lệnh CLI để xác nhận dữ liệu thực sự tồn tại trên topic:
     ```bash
     docker exec -it broker kafka-console-consumer --bootstrap-server broker:29092 --topic users_created --from-beginning
     ```

5. **Đảm bảo format dữ liệu:**
   - Đảm bảo rằng dữ liệu được gửi lên Kafka là JSON hoặc một format hỗ trợ:
     - Sửa script `kafka-stream.py` nếu cần để định dạng dữ liệu:
       ```python
       producer.send('users_created', json.dumps(res).encode('utf-8'))
       ```

---

Nếu vẫn không khắc phục được, cung cấp log của Control Center và Kafka để hỗ trợ thêm!

Dựa trên thông tin bạn cung cấp và hình ảnh, vấn đề có thể xảy ra do một số nguyên nhân sau:

---

### **Nguyên nhân 1: Control Center không hiển thị dữ liệu cũ**
- **Control Center chỉ hiển thị dữ liệu được gửi vào sau khi trang được mở**. Dữ liệu đã có sẵn trong topic (xác minh bằng `kafka-console-consumer`) sẽ không được hiển thị nếu chúng đã được gửi trước đó.

#### **Giải pháp:**
1. **Gửi thêm dữ liệu mới vào topic:**
   - Chạy lại `kafka-stream.py` để gửi thêm dữ liệu vào Kafka.
2. **Refresh Control Center:**
   - Mở lại tab `Messages` trong Control Center sau khi dữ liệu mới đã được gửi.

---

### **Nguyên nhân 2: Không có Schema được xác định cho topic**
- **Control Center có thể không hiển thị dữ liệu nếu không có schema (Avro, JSON, etc.) được cấu hình trong Schema Registry**. Dữ liệu được gửi dưới dạng raw binary sẽ không hiển thị trên giao diện.

#### **Giải pháp:**
1. **Kiểm tra định dạng dữ liệu:**
   - Trong script `kafka-stream.py`, kiểm tra định dạng dữ liệu gửi lên Kafka. Đảm bảo rằng dữ liệu được gửi dưới dạng JSON.
   - Dòng gửi dữ liệu phải giống như:
     ```python
     producer.send('users_created', json.dumps(data).encode('utf-8'))
     ```

2. **Xác minh Schema Registry:**
   - Kiểm tra Schema Registry bằng cách truy cập:
     ```
     http://localhost:8081
     ```
   - Đảm bảo rằng topic `users_created` có schema tương ứng được đăng ký.

---

### **Nguyên nhân 3: Consumer đang tiêu thụ dữ liệu**
- **Một Consumer khác có thể đang tiêu thụ hết dữ liệu từ topic `users_created`**, khiến Control Center không thể hiển thị dữ liệu nào trong tab `Messages`.

#### **Giải pháp:**
1. **Kiểm tra consumer:**
   - Chạy lệnh kiểm tra các consumer group đang sử dụng topic `users_created`:
     ```bash
     docker exec -it broker kafka-consumer-groups --bootstrap-server broker:29092 --describe --all-groups
     ```
   - Xem các thông tin như `OFFSET` và `LAG` để kiểm tra dữ liệu đã được tiêu thụ hay chưa.

2. **Tạm dừng consumer khác:**
   - Nếu có consumer đang tiêu thụ dữ liệu, tạm dừng hoặc dừng consumer đó để xác minh.

---

### **Nguyên nhân 4: Offset không được đặt đúng**
- **Control Center có thể không tự động đọc từ offset `earliest`.** Nếu offset mặc định được đặt là `latest`, nó sẽ không hiển thị dữ liệu đã được gửi trước đó.

#### **Giải pháp:**
1. **Đặt lại offset về `earliest`:**
   - Trong Control Center, sử dụng tùy chọn `Jump to offset` và chọn offset `0`.
   - Hoặc chạy lệnh sau để kiểm tra:
     ```bash
     docker exec -it broker kafka-console-consumer \
       --bootstrap-server broker:29092 \
       --topic users_created \
       --from-beginning
     ```

---

### **Nguyên nhân 5: Vấn đề hiển thị trong Control Center**
- **Control Center có thể bị lỗi caching hoặc refresh chậm.**

#### **Giải pháp:**
1. **Xóa cache trình duyệt:**
   - Refresh trang hoặc thử mở lại Control Center trong trình duyệt ẩn danh.
2. **Kiểm tra log Control Center:**
   - Chạy lệnh kiểm tra log:
     ```bash
     docker logs control-center
     ```
   - Xem các lỗi liên quan đến kết nối hoặc hiển thị.

---

### **Tóm tắt các bước cần thực hiện**
1. **Gửi thêm dữ liệu mới vào Kafka và refresh Control Center.**
2. **Đảm bảo dữ liệu được gửi dưới dạng JSON hoặc Avro.**
3. **Kiểm tra các consumer khác đang tiêu thụ topic.**
4. **Đặt offset về `earliest` để xem toàn bộ dữ liệu.**
5. **Kiểm tra log của Control Center để phát hiện lỗi.**

Nếu vẫn không khắc phục được, bạn có thể chia sẻ thêm thông tin log từ Control Center hoặc Kafka để phân tích chi tiết hơn!

## Check thử 
```
2. **Xác minh Schema Registry:**
   - Kiểm tra Schema Registry bằng cách truy cập:
     ```
     http://localhost:8081
     ```
   - Đảm bảo rằng topic `users_created` có schema tương ứng được đăng ký.

---

### **Nguyên nhân 3: Consumer đang tiêu thụ dữ liệu**
- **Một Consumer khác có thể đang tiêu thụ hết dữ liệu từ topic `users_created`**, khiến Control Center không thể hiển thị dữ liệu nào trong tab `Messages`.

#### **Giải pháp:**
1. **Kiểm tra consumer:**
   - Chạy lệnh kiểm tra các consumer group đang sử dụng topic `users_created`:
     ```bash
     docker exec -it broker kafka-consumer-groups --bootstrap-server broker:29092 --describe --all-groups
     ```
   - Xem các thông tin như `OFFSET` và `LAG` để kiểm tra dữ liệu đã được tiêu thụ hay chưa.

2. **Tạm dừng consumer khác:**
   - Nếu có consumer đang tiêu thụ dữ liệu, tạm dừng hoặc dừng consumer đó để xác minh.

```

### Phân tích vấn đề từ dữ liệu bạn cung cấp


#### 2. **Không thấy Schema tại `localhost:8081`:**
- Giao diện Schema Registry tại `http://localhost:8081` trả về `{}`.
- Điều này cho thấy không có schema nào được đăng ký cho topic `users_created`.

**Hệ quả:**
- Nếu topic được ghi dữ liệu dưới dạng `JSON` hoặc `Avro`, nhưng không đăng ký schema, Control Center sẽ không biết cách diễn giải dữ liệu, và do đó không hiển thị dữ liệu trong tab **Messages**.

---

#### 3. **Consumer group `_confluent-controlcenter-7-4-0-lastProduceTimeConsumer` không có thành viên hoạt động:**
- **Thông báo:** `Consumer group '_confluent-controlcenter-7-4-0-lastProduceTimeConsumer' has no active members.`
- Điều này cho thấy consumer group không có ứng dụng nào đang hoạt động để tiêu thụ dữ liệu từ topic `users_created`.

---

### Giải pháp để khắc phục



#### 2. **Đăng ký Schema cho topic `users_created`:**
- Nếu dữ liệu được gửi vào topic dưới dạng JSON hoặc Avro, cần đăng ký schema.
- Đăng ký schema qua lệnh `curl` hoặc REST API của Schema Registry:
  ```bash
  curl -X POST -H "Content-Type: application/vnd.schemaregistry.v1+json" \
       --data '{
         "schema": "{\"type\":\"record\",\"name\":\"User\",\"fields\":[{\"name\":\"first_name\",\"type\":\"string\"},{\"name\":\"last_name\",\"type\":\"string\"},{\"name\":\"gender\",\"type\":\"string\"},{\"name\":\"address\",\"type\":\"string\"},{\"name\":\"post_code\",\"type\":\"string\"},{\"name\":\"email\",\"type\":\"string\"},{\"name\":\"username\",\"type\":\"string\"},{\"name\":\"dob\",\"type\":\"string\"},{\"name\":\"registered_date\",\"type\":\"string\"},{\"name\":\"phone\",\"type\":\"string\"},{\"name\":\"picture\",\"type\":\"string\"}]}"
       }' http://localhost:8081/subjects/users_created-value/versions
  ```

**Lưu ý:**
- Thay schema trong JSON trên bằng định dạng thực tế của dữ liệu bạn đang gửi.

---

#### 3. **Khởi động lại consumer group cho topic `users_created`:**
- Đảm bảo consumer group `_confluent-controlcenter-7-4-0-lastProduceTimeConsumer` có thành viên hoạt động.
- Nếu cần, restart `Control Center`:
  ```bash
  docker-compose restart control-center
  ```
- Check lại Control Center


---



### fix 1 hồi thì là do cần phải **Đăng ký Schema cho topic `users_created`:**
```
curl -X POST -H "Content-Type: application/vnd.schemaregistry.v1+json" \
     --data '{
       "schema": "{\"type\":\"record\",\"name\":\"User\",\"fields\":[{\"name\":\"first_name\",\"type\":\"string\"},{\"name\":\"last_name\",\"type\":\"string\"},{\"name\":\"gender\",\"type\":\"string\"},{\"name\":\"address\",\"type\":\"string\"},{\"name\":\"post_code\",\"type\":\"string\"},{\"name\":\"email\",\"type\":\"string\"},{\"name\":\"username\",\"type\":\"string\"},{\"name\":\"dob\",\"type\":\"string\"},{\"name\":\"registered_date\",\"type\":\"string\"},{\"name\":\"phone\",\"type\":\"string\"},{\"name\":\"picture\",\"type\":\"string\"}]}"
     }' http://localhost:8081/subjects/users_created-value/versions

```